In [103]:
from collections import defaultdict
import copy
import logging
import os
import json
import time

import torch.utils.data
from PIL import Image

import openpifpaf
import numpy as np

from pycocotools.coco import COCO

In [104]:
class Json_Updating: 

    def __init__(self):
        print('json file init')
        self.json_file = {}


    def initiate_json(self, name = '', id = 1, skeleton = [], supercategory = '', keypoints = []):
        """
        Initiate json file: one for training phase and another one for validation.
        """
        self.json_file["info"] = dict(url="https://github.com/vita-epfl/openpifpaf",
                                    date_created=time.strftime("%a, %d %b %Y %H:%M:%S +0000", time.localtime()),
                                    description="Conversion of ApolloCar3D dataset into MS-COCO format")
        self.json_file["categories"] = [dict(name= name,  # Category name
                                            id = id,  # Id of category
                                            skeleton= skeleton,  # Skeleton connections (check constants.py)
                                            supercategory= supercategory,  # Same as category if no supercategory
                                            keypoints= keypoints)]  # Keypoint names
        self.json_file["images"] = []  # Empty for initialization
        self.json_file["annotations"] = []  # Empty for initialization


    def process_image(self, images_anns):
        """
        Update image field in json file
        """
        # ------------------
        # Add here your code
        # -------------------
        self.json_file["images"].append(images_anns)
        # self.json_file["images"].append({
        #     'coco_url': "unknown",
        #     'file_name': '',  # Image name
        #     'id': 0,  # Image id
        #     'license': 1,  # License type
        #     'date_captured': "unknown",  
        #     'width': 0,  # Image width (pixels)
        #     'height': 0})  # Image height (pixels)


    def process_annotation(self, image_id, category_id, iscrowd, id, area, bbox, num_keypoints, keypoints, segmentation):
        """
        Process and include in the json file a single annotation (instance) from a given image
        """
        # ------------------
        # Add here your code
        # -------------------
        self.json_file["annotations"].append({
            'image_id': image_id,  # Image id
            'category_id': category_id,  # Id of the category (like car or person)
            'iscrowd': iscrowd,  # 1 to mask crowd regions, 0 if the annotation is not a crowd annotation
            'id': id,  # Id of the annotations
            'area': area,  # Bounding box area of the annotation (width*height)
            'bbox': bbox,  # Bounding box  coordinates (x0, y0, width, heigth), where x0, y0 are the left corner
            'num_keypoints': num_keypoints,  # number of keypoints
            'keypoints': keypoints,  # Flattened list of keypoints [x, y, visibility, x, y, visibility, .. ]
            'segmentation': segmentation})  # To add a segmentation of the annotation, empty otherwise
        
        # self.json_file["annotations"].append({
        #     'image_id': 0,  # Image id
        #     'category_id': 1,  # Id of the category (like car or person)
        #     'iscrowd': 0,  # 1 to mask crowd regions, 0 if the annotation is not a crowd annotation
        #     'id': 0,  # Id of the annotations
        #     'area': 0,  # Bounding box area of the annotation (width*height)
        #     'bbox': [],  # Bounding box  coordinates (x0, y0, width, heigth), where x0, y0 are the left corner
        #     'num_keypoints': 0,  # number of keypoints
        #     'keypoints': [],  # Flattened list of keypoints [x, y, visibility, x, y, visibility, .. ]
        #     'segmentation': []})  # To add a segmentation of the annotation, empty otherwise


    def save_json_files(self):

        name = 'keypoints2box_test.json'
        path_json = 'keypoints2box_test.json'
        with open(path_json, 'w') as outfile:
                json.dump(self.json_file, outfile)

# Test: initialisation

In [106]:
ann_path = 'data-mscoco/small_dataset/train/person_keypoints_small_train2017_understand.json'

coco = COCO(ann_path)
json_file = Json_Updating()

category_ids = coco.getCatIds()

json_file.initiate_json(name = 'person', id = category_ids[0], skeleton = [[1, 2], [1, 3], [1, 4], [1, 5], [2, 3], [2, 4], [3, 5], [5, 4]], supercategory = 'person', keypoints = ['box_center','box_left_up_corner','box_right_up_corner','box_left_down_corner','box_right_down_corner'])

ids = coco.getImgIds(catIds=category_ids)
#print(ids)

for image_id in ids:

    # Get ids from annotation and categories 
    ann_ids = coco.getAnnIds(imgIds=image_id, catIds=category_ids)
    #cat_ids = coco.getCatIds()

    ########### Images extraction ###########

    # Extract the data of annotation, images and categories
    images = coco.loadImgs(image_id)
    
    # Write the images anns in the json
    json_file.process_image(images[0])
    

    ########### annotation extraction ###########

    # load anns from the json file
    anns = coco.loadAnns(ann_ids) # load segemntation data
    #anns = [ann for ann in anns if not ann.get('iscrowd')]
    
    # extract kps from the anns
    kp_anns = [ann for ann in anns if 'keypoints' in ann and any(v > 0.0 for v in ann['keypoints'][2::3])] # Check si keypoint présents et > 0
    kp_anns = [ann.get('keypoints') for ann in kp_anns] #Load the kp

    # extract boxes from the anns
    box_anns = [ann for ann in anns if 'bbox' in ann and any(v > 0.0 for v in ann['bbox'])] # Check si il y a au moins 1 bbox > 0
    box_anns = [ann.get('bbox') for ann in box_anns]

    # Create new boxes
    if len(box_anns) > 0:
        new_box_anns = []
        box = box_anns[0]
        b0, b1, w, h = box[0], box[1], box[2], box[3]
        box_vals = [round((b0 + w)/2.0, 2), round((b1+h)/2.0, 2), 2., round(b0,2), round(b1,2), 2., round(b0+w,2), round(b1,2), 2., round(b0,2), round(b1+h,2), 2., round(b0+w,2), round(b1+h,2), 2.]
        for i in range(len(box_vals)): new_box_anns.append(box_vals[i])
    else : 
        new_box_anns = [0 for i in range(15)] # Put new boxes to 0 if there is no bounding boxes
        box_anns = [[0,0,0,0]]

    # extract other non changing annotations to copy inside the json file from the anns
    segm_anns = [ann.get('segmentation') for ann in anns]
    num_keypoints_anns = len(new_box_anns)/3
    area_anns = [ann.get('area') for ann in anns]
    iscrowd_anns = [ann.get('iscrowd') for ann in anns]
    id_anns = [ann.get('id') for ann in anns]
    category_id_anns = [ann.get('category_id') for ann in anns]


    json_file.process_annotation(image_id, category_id_anns[0], iscrowd_anns[0], id_anns[0], area_anns[0], box_anns[0], num_keypoints_anns, new_box_anns, segm_anns[0])


json_file.save_json_files()

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
json file init


### FONCTIONNE !! 

# Test: Transform Boxes

In [78]:
ann_path = 'data-mscoco/small_dataset/train/person_keypoints_small_train2017_understand.json'

coco = COCO(ann_path)
json_file = Json_Updating()

# Category that are taking into consideration
#category_ids = [1]
category_ids = coco.getCatIds()
#cats = coco.loadCats(category_ids)
# Boucle pour category ID

ids = coco.getImgIds(catIds=category_ids)
#print(ids)

for image_id in ids:

    # Get ids from annotation and categories 
    ann_ids = coco.getAnnIds(imgIds=image_id, catIds=category_ids)
    #cat_ids = coco.getCatIds()

    # Extract the data of annotation, images and categories
    
    images = coco.loadImgs(image_id)

    #print(ann_ids)
    anns = coco.loadAnns(ann_ids) # load segemntation data
    #print(anns)
    anns = [ann for ann in anns if not ann.get('iscrowd')]
    #print(anns)
    kp_anns = [ann for ann in anns if 'keypoints' in ann and any(v > 0.0 for v in ann['keypoints'][2::3])] # Check si keypoint présents et > 0
    kp_anns = [ann.get('keypoints') for ann in kp_anns] #Load the kp
    box_anns = [ann for ann in anns if 'bbox' in ann and any(v > 0.0 for v in ann['bbox'])] # Check si il y a au moins 1 bbox > 0
    #print(box_anns)
    box_anns = [ann.get('bbox') for ann in box_anns]
    #print(box_anns)

    # Create new boxes
    if len(box_anns) > 0:
        new_box_anns = []
        box = box_anns[0]
        b0, b1, w, h = box[0], box[1], box[2], box[3]
        box_vals = [round((b0 + w)/2.0, 2), round((b1+h)/2.0, 2), 2., round(b0,2), round(b1,2), 2., round(b0+w,2), round(b1,2), 2., round(b0,2), round(b1+h,2), 2., round(b0+w,2), round(b1+h,2), 2.]
        for i in range(len(box_vals)): new_box_anns.append(box_vals[i])
    else : new_box_anns = [0 for i in range(15)] # Put new boxes to 0 if there is no bounding boxes

    #print(new_box_anns)



loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
json file init
